* Read in and transform parquets into single files and load them
* Data from openTargets - https://platform.opentargets.org/downloads


ALSO - ICD10 occurences, using table from UKBB and nadav Rap -
* `ICD10_counts.csv`
```
a table with counts (if you want frequencies, divide by 502616) of ICD10 codes. I created two columns, one that counts occurrences in the field 41202, the other is any of 41202, 40006, 41204
```
* Getting the ICD codes from diseases file (json or parquet):
* https://community.opentargets.org/t/how-to-get-icd-codes-for-diseases/900/3
```
import pandas as pd
path = 'some path to files'
ref_filter = r"^ICD[9,10].+"
raw_df: pd.DataFrame = pd.read_parquet(path, columns=['id', 'dbXRefs'])
id_and_xref_df: pd.DataFrame = raw_df.explode('dbXRefs').astype('string')
id_and_icd_df: pd.DataFrame = id_and_xref_df[id_and_xref_df["dbXRefs"].str.match(
    ref_filter)]
```

In [1]:
import pandas as pd
import os,sys
import numpy as np


In [2]:
# !pip install pyarrow fastparquet pandas -U

In [3]:
raw_data_directory = "."

In [4]:
dirs = [x[0] for x in os.walk(raw_data_directory,topdown=False) if ("/proc" not in x)]
bad_res = ['.','./.ipynb_checkpoints','./proc']
dirs = [x for x in dirs if not any(x == c for c in bad_res)]


### does excluding these mean no double counting of evidence????
sub_ev = [ './evidence/sourceId=progeny',
 './evidence/sourceId=chembl',
 './evidence/sourceId=slapenrich',
 './evidence/sourceId=cancer_gene_census',
 './evidence/sourceId=intogen',
 './evidence/sourceId=expression_atlas',
 './evidence/sourceId=europepmc',
 './evidence/sourceId=eva_somatic',
 './evidence/sourceId=genomics_england',
 './evidence/sourceId=crispr',
 './evidence/sourceId=sysbio',
 './evidence/sourceId=impc',
 './evidence/sourceId=reactome',
 './evidence/sourceId=gene_burden',
 './evidence/sourceId=gene2phenotype',
 './evidence/sourceId=cancer_biomarkers',
 './evidence/sourceId=ot_genetics_portal',
 './evidence/sourceId=orphanet',
 './evidence/sourceId=uniprot_literature',
 './evidence/sourceId=eva',
 './evidence/sourceId=clingen',
 './evidence/sourceId=uniprot_variants',
#          './fda/significantAdverseDrugReactions', # unneeded, and included in /fda - ?
          
#           './evidence', # save time on rerun
         ]
dirs = [x for x in dirs if not any(x == c for c in sub_ev)]

dirs

['./go',
 './evidence',
 './diseaseToPhenotype',
 './targets',
 './associationByDatasourceDirect',
 './associationByDatatypeDirect',
 './fda/significantAdverseDrugReactions',
 './fda',
 './mechanismOfAction',
 './indication',
 './associationByOverallDirect',
 './associationByOverallIndirect',
 './diseases',
 './drugWarnings',
 './mousePhenotypes',
 './molecule',
 './hpo',
 './interaction']

In [5]:
# dirs = [
# ## todo: rerun with evidences
#         './go',
        
#  './evidence',
#  './evidence/sourceId=chembl',
#  './evidence/sourceId=cancer_gene_census',
#  './evidence/sourceId=europepmc',
#  './evidence/sourceId=crispr',
#  './evidence/sourceId=cancer_biomarkers',
#  './evidence/sourceId=clingen',
#  './diseaseToPhenotype',
#  './targets',
#  './associationByDatasourceDirect',
#  './fda',
#  './fda/significantAdverseDrugReactions',
#  './mechanismOfAction',
#  './indication',
#  './associationByOverallDirect',
#  './associationByOverallIndirect',
#  './diseases',
#  './mousePhenotypes',
#  './molecule',
#  './hpo',
#  './interaction'
#        ]

# dirs = [
# #     './drugWarnings',
#  './mousePhenotypes',
#  './molecule',
#  './hpo',
#  './interaction']


dirs = [
#     './drugWarnings',
 './diseases',
# './diseaseToPhenotype',
# #      './associationByOverallDirect',
#     "./go",
#  './targets',
#     './hpo',
]

"Q600 is indeed Q60.0, and those is true for all other codes.

In [6]:
df_icd_counts_ukbb = pd.read_csv("ICD10_counts.csv")
df_icd_counts_ukbb["ICD10"] = df_icd_counts_ukbb["ICD10"].str.strip()
df_icd_counts_ukbb.set_index("ICD10",inplace=True)
df_icd_counts_ukbb#["ICD10"]

,f41202,All
ICD10,,
C457,37,52
S229,2,2
M628,76,184
Y733,0,5
T294,1,1
...,...,...
R401,15,36
X126,0,1
I050,58,130


```
"Q600 is indeed Q60.0, and those is true for all other codes."
```

In [7]:
import pandas as pd
path = './diseases'
ref_filter = r"^ICD[10].+" #r"^ICD[9,10].+"
raw_df: pd.DataFrame = pd.read_parquet(path, columns=['id', 'dbXRefs'])
id_and_xref_df: pd.DataFrame = raw_df.explode('dbXRefs').astype('string')
id_and_icd_df: pd.DataFrame = id_and_xref_df[id_and_xref_df["dbXRefs"].str.match(
    ref_filter)]

display(id_and_icd_df.head(2))
# ## DROP CM/CMS cases
# id_and_icd_df = id_and_icd_df.loc[~id_and_icd_df["dbXRefs"].str.startswith("ICD10CM")]

# note : this'll also include "ICD10CM:" if not dropped before
id_and_icd_df["dbXRefs"] =id_and_icd_df["dbXRefs"].str.split(":",expand=True)[1]
## get ~"3 digit" codes only
id_and_icd_df["ICD10"] =id_and_icd_df["dbXRefs"].str.split(".",expand=True)[0].str.strip()
## alt col without dot and as long number
id_and_icd_df["ICD10_2"] =id_and_icd_df["dbXRefs"].str.replace(r".","",regex=False).str.strip()
print(id_and_icd_df.nunique())
id_and_icd_df

,id,dbXRefs
10,MONDO_0007097,ICD10CM:E85.1
11,MONDO_0007216,ICD10CM:Q73.8


id         10110
dbXRefs     2949
ICD10       1231
ICD10_2     2949
dtype: int64


/var/folders/hl/bb0z87yd5pzfcp81jc8662kr0000gn/T/ipykernel_61553/1169054070.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  id_and_icd_df["dbXRefs"] =id_and_icd_df["dbXRefs"].str.split(":",expand=True)[1]
/var/folders/hl/bb0z87yd5pzfcp81jc8662kr0000gn/T/ipykernel_61553/1169054070.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  id_and_icd_df["ICD10"] =id_and_icd_df["dbXRefs"].str.split(".",expand=True)[0].str.strip()
/var/folders/hl/bb0z87yd5pzfcp81jc8662kr0000gn/T/ipykernel_61553/1169054070.py:18:

,id,dbXRefs,ICD10,ICD10_2
10,MONDO_0007097,E85.1,E85,E851
11,MONDO_0007216,Q73.8,Q73,Q738
12,MONDO_0007604,Q87.8,Q87,Q878
13,MONDO_0008291,Q82.8,Q82,Q828
14,MONDO_0008695,E78.6,E78,E786
...,...,...,...,...
23066,Orphanet_294939,Q69.1,Q69,Q691
23068,Orphanet_352734,E70.3,E70,E703
23069,Orphanet_721,D69.1,D69,D691
23070,Orphanet_93322,Q72.5,Q72,Q725


In [8]:
id_and_icd_df = id_and_icd_df.merge(df_icd_counts_ukbb,on=["ICD10"],how="left")
id_and_icd_df = id_and_icd_df.merge(df_icd_counts_ukbb,left_on=["ICD10_2"],right_on=["ICD10"],how="left")
id_and_icd_df["All_ICD_counts"] = id_and_icd_df["All_x"].fillna(id_and_icd_df["All_y"])
id_and_icd_df["f41202_ICD_counts"] = id_and_icd_df["f41202_x"].fillna(id_and_icd_df["f41202_y"])
# normalized %)
id_and_icd_df = id_and_icd_df[['id', 'dbXRefs', 'ICD10','All_ICD_counts', 'f41202_ICD_counts']]

id_and_icd_df["ICD10_all_freq"] = 100*id_and_icd_df["All_ICD_counts"]/502616
id_and_icd_df

,id,dbXRefs,ICD10,All_ICD_counts,f41202_ICD_counts,ICD10_all_freq
0,MONDO_0007097,E85.1,E85,2.0,1.0,0.000398
1,MONDO_0007216,Q73.8,Q73,NaN,NaN,NaN
2,MONDO_0007604,Q87.8,Q87,28.0,5.0,0.005571
3,MONDO_0008291,Q82.8,Q82,86.0,52.0,0.017110
4,MONDO_0008695,E78.6,E78,27.0,0.0,0.005372
...,...,...,...,...,...,...
12011,Orphanet_294939,Q69.1,Q69,NaN,NaN,NaN
12012,Orphanet_352734,E70.3,E70,15.0,2.0,0.002984
12013,Orphanet_721,D69.1,D69,14.0,5.0,0.002785
12014,Orphanet_93322,Q72.5,Q72,NaN,NaN,NaN


In [9]:
id_and_icd_df.loc[id_and_icd_df["All_ICD_counts"].isna()].drop_duplicates(subset=["ICD10"])

,id,dbXRefs,ICD10,All_ICD_counts,f41202_ICD_counts,ICD10_all_freq
1,MONDO_0007216,Q73.8,Q73,NaN,NaN,NaN
9,MONDO_0009061,E84,E84,NaN,NaN,NaN
14,MONDO_0010080,G23.2,G23,NaN,NaN,NaN
24,MONDO_0008322,Q77.8,Q77,NaN,NaN,NaN
33,MONDO_0011348,Q69.0,Q69,NaN,NaN,NaN
...,...,...,...,...,...,...
11040,Orphanet_797,D86,D86,NaN,NaN,NaN
11090,Orphanet_93976,Q16.0,Q16,NaN,NaN,NaN
11594,Orphanet_98602,H06,H06,NaN,NaN,NaN
11643,Orphanet_294944,M21,M21,NaN,NaN,NaN


#### Only ~ 75% coverage

In [10]:
id_and_icd_df = id_and_icd_df.dropna(subset=["All_ICD_counts","f41202_ICD_counts"],how="all",axis=0)
print(id_and_icd_df.dropna(axis=0,thresh=5).nunique())
id_and_icd_df

id                   8445
dbXRefs              1518
ICD10                 663
All_ICD_counts        441
f41202_ICD_counts     339
ICD10_all_freq        441
dtype: int64


,id,dbXRefs,ICD10,All_ICD_counts,f41202_ICD_counts,ICD10_all_freq
0,MONDO_0007097,E85.1,E85,2.0,1.0,0.000398
2,MONDO_0007604,Q87.8,Q87,28.0,5.0,0.005571
3,MONDO_0008291,Q82.8,Q82,86.0,52.0,0.017110
4,MONDO_0008695,E78.6,E78,27.0,0.0,0.005372
5,MONDO_0008796,Q87.8,Q87,28.0,5.0,0.005571
...,...,...,...,...,...,...
12006,Orphanet_93457,Q73.1,Q73,1.0,0.0,0.000199
12007,Orphanet_95699,E25.0,E25,19.0,9.0,0.003780
12010,Orphanet_293925,Q87.5,Q87,2.0,0.0,0.000398
12012,Orphanet_352734,E70.3,E70,15.0,2.0,0.002984


In [13]:
id_and_icd_df.to_csv("disease_icd10_counts_v1.csv",index=False)

In [12]:
# id_and_icd_df.loc[id_and_icd_df["ICD10"].isin(df_icd_counts_ukbb["ICD10"])]

In [118]:
print("Not in:")
id_and_icd_df.loc[~id_and_icd_df["ICD10"].isin(df_icd_counts_ukbb["ICD10"])].drop_duplicates(subset=["ICD10"])

Not in:


KeyError: 'ICD10'

In [61]:
# df_icd_counts_ukbb.loc[df_icd_counts_ukbb["ICD10"].str.startswith("Q0",na=False)] # no matches for Q00
# df_icd_counts_ukbb.loc[df_icd_counts_ukbb["ICD10"].str.startswith("Q82",na=False)]
# df_icd_counts_ukbb.loc[df_icd_counts_ukbb["ICD10"].str.startswith("N98",na=False)]
### N981 is probably actually N98.1 - weird coding error? ? 

df_icd_counts_ukbb.loc[df_icd_counts_ukbb.index.str.startswith("Q75",na=False)]
## 

,f41202,All
ICD10,,
Q750,1,3
Q751,2,2
Q754,2,6
Q759,0,2
Q758,1,4


In [20]:
df_icd_counts_ukbb.loc[df_icd_counts_ukbb["ICD10"].str.startswith("Q60",na=False)]
## Q603 is a ICD10-CM code
## is Q603 a code , or is it actually Q60.3 without dot?

,ICD10,f41202,All
1234,Q600,0,109
1278,Q603,0,10
1420,Q605,0,1
4256,Q606,0,2
8504,Q602,1,31


In [8]:
try:
    os.mkdir("proc")
except: ()

In [9]:
# https://stackoverflow.com/questions/68814219/how-to-fix-arrowinvalid-could-not-convert-x-y-with-type-tuple

In [10]:
# print(len(bad_datas))
# print(bad_datas) # './evidence', './evidence/sourceId=europepmc', './molecule'

In [11]:
# df.iloc[0:2,65:67]

In [12]:
# df.iloc[:,65:67].tail() # 	targetModulation	textMiningSentences

#### munge dtypes to not use list? 
* Loses info, but easier to upload later
* parse as str / categorical

In [13]:
%%time
bad_datas = []
for i in dirs:
    print(i.strip("./"))
    df = pd.read_parquet(i) 
    ## use with replace? slow! ,na_values=["None",[],"[]"] 
    df.drop(columns=["obsoleteTerms",
                     "dbXRefs", # ? keep or not? e.g. for hpo 
                     "crossReferences","urls","references",
                     "obsoleteNames","obsolete_terms",
                     "obsoleteSymbols",
#                      "textMiningSentences","country",
                     "betaConfidenceIntervalLower","betaConfidenceIntervalUpper",
                     "textMiningSentences", ## causes error with array shape, maybe large?
#                      "chemblIds"
                     "tep.url","dir0","indirectLocationIds",
                     "ontology.leaf",
                     "ontology.sources.url"
                    ],errors="ignore",inplace=True)   
    display(df) 
    ### SB can't handle list types - convert to categorical or string.. 
    total_shape = df.shape[0]
    print(total_shape)
    for c in df.select_dtypes("O").columns:
        df[c] = df[c].astype(str).replace("None",np.NaN)
#         df = df.replace("None",np.NaN)
        if df[c].nunique()< (total_shape/10):
            if total_shape>1234567:
                print(c)
            df[c] = df[c].astype("category")

    try: df.dropna(axis=1,how="all",inplace=True)        
    except: ()
    try: df.drop_duplicates(inplace=True)
    except: df = df.loc[df.astype(str).drop_duplicates().index] # drop dupes when there's lists
    display(df.info(memory_usage=True))
    
    try:
        df.to_parquet("./proc/"+i.strip("./").replace(r"/","_")+".parquet",index=False)
#     except:
#         try:
#             df.infer_objects().to_parquet("./proc/"+i.strip("./").replace(r"/","_")+".parquet",index=False,
#                                          engine="pyarrow")
    except:
        print("Failed to save",i.strip("./"))
        bad_datas.append(i)

diseases


,id,code,description,name,directLocationIds,parents,sko,synonyms,ancestors,descendants,children,therapeuticAreas,ontology
0,DOID_7551,http://purl.obolibrary.org/obo/DOID_7551,A primary bacterial infectious disease that is...,gonorrhea,None,"[EFO_0003955, MONDO_0000314]",None,"{'hasBroadSynonym': None, 'hasExactSynonym': [...","[EFO_0000512, EFO_0009663, MONDO_0100336, OTAR...",[],[],"[MONDO_0100336, OTAR_0000017, EFO_0005741, EFO...","{'isTherapeuticArea': False, 'leaf': True, 'so..."
1,HP_0000031,http://purl.obolibrary.org/obo/HP_0000031,The presence of inflammation of the epididymis.,Epididymitis,None,"[HP_0012649, HP_0000022]",None,None,"[HP_0000022, HP_0000078, EFO_0000651, HP_00126...",[],[],[EFO_0000651],"{'isTherapeuticArea': False, 'leaf': True, 'so..."
2,HP_0000857,http://purl.obolibrary.org/obo/HP_0000857,None,Neonatal insulin-dependent diabetes mellitus,None,"[EFO_0000400, HP_0001952, HP_0000818]",None,None,"[HP_0000818, HP_0000118, OTAR_0000020, HP_0011...",[],[],"[OTAR_0000020, EFO_0000651, EFO_0001379, EFO_0...","{'isTherapeuticArea': False, 'leaf': True, 'so..."
3,HP_0001693,http://purl.obolibrary.org/obo/HP_0001693,Pattern of blood flow in the heart that deviat...,Cardiac shunt,None,[HP_0011028],None,None,"[EFO_0000651, HP_0011028, HP_0000118, HP_00110...",[HP_0012382],[HP_0012382],[EFO_0000651],"{'isTherapeuticArea': False, 'leaf': False, 's..."
4,HP_0001730,http://purl.obolibrary.org/obo/HP_0001730,A progressive form of hearing impairment.,Progressive hearing impairment,None,[HP_0000364],None,"{'hasBroadSynonym': None, 'hasExactSynonym': [...","[HP_0000364, EFO_0000651, HP_0000598, HP_0000118]",[HP_0000408],[HP_0000408],[EFO_0000651],"{'isTherapeuticArea': False, 'leaf': False, 's..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23069,Orphanet_721,http://www.orpha.net/ORDO/Orphanet_721,Gray platelet syndrome (GPS) is a rare inherit...,Gray platelet syndrome,None,[Orphanet_98455],None,"{'hasBroadSynonym': None, 'hasExactSynonym': [...","[Orphanet_183654, OTAR_0000018, Orphanet_98454...",[],[],"[OTAR_0000018, EFO_0005803]","{'isTherapeuticArea': False, 'leaf': True, 'so..."
23070,Orphanet_93322,http://www.orpha.net/ORDO/Orphanet_93322,Tibial hemimelia is a rare congenital anomaly ...,Tibial hemimelia,None,[Orphanet_2130],None,"{'hasBroadSynonym': None, 'hasExactSynonym': [...","[EFO_0004260, Orphanet_404571, OTAR_0000006, O...","[Orphanet_295079, Orphanet_295077]","[Orphanet_295077, Orphanet_295079]","[OTAR_0000006, OTAR_0000018]","{'isTherapeuticArea': False, 'leaf': False, 's..."
23071,Orphanet_93925,http://www.orpha.net/ORDO/Orphanet_93925,Alobar holoprosencephaly is the most severe cl...,Alobar holoprosencephaly,None,[Orphanet_2162],None,None,"[Orphanet_183763, Orphanet_183533, OTAR_000001...",[],[],"[OTAR_0000018, EFO_0001379]","{'isTherapeuticArea': False, 'leaf': True, 'so..."
23072,Orphanet_98539,http://www.orpha.net/ORDO/Orphanet_98539,None,Early-onset ataxia with dementia,None,[Orphanet_98538],None,None,"[Orphanet_71859, Orphanet_183500, Orphanet_276...","[Orphanet_247815, EFO_0009016, EFO_0010249, EF...",[Orphanet_1172],"[OTAR_0000018, EFO_0000618]","{'isTherapeuticArea': False, 'leaf': False, 's..."


23074
<class 'pandas.core.frame.DataFrame'>
Int64Index: 23074 entries, 0 to 23073
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   id                 23074 non-null  object  
 1   code               23074 non-null  object  
 2   description        16949 non-null  object  
 3   name               23074 non-null  object  
 4   directLocationIds  12 non-null     category
 5   parents            23074 non-null  object  
 6   sko                1 non-null      category
 7   synonyms           16339 non-null  object  
 8   ancestors          23074 non-null  object  
 9   descendants        23074 non-null  object  
 10  children           23074 non-null  object  
 11  therapeuticAreas   23074 non-null  category
 12  ontology           23074 non-null  object  
dtypes: category(3), object(10)
memory usage: 2.1+ MB


None

diseaseToPhenotype


...

hpo


,description,id,name,namespace,parents
0,None,BFO_0000019,quality,[bfo],"[b91618, BFO_0000020]"
1,None,BFO_0000030,object,[bfo],[BFO_0000040]
2,An independent continuant that is spatially ex...,BFO_0000040,material entity,[bfo],[BFO_0000004]
3,Elementary particle not affected by the strong...,CHEBI_10545,electron,[chebi_ontology],[CHEBI_36338]
4,A dicarboxylic acid dianion resulting from the...,CHEBI_11596,2-hydroxyglutarate(2-),[chebi_ontology],"[b67169, CHEBI_28965, CHEBI_18059, b67170, CHE..."
...,...,...,...,...,...
26365,A protein coding gene OPLAH in human.,HGNC_8149,OPLAH (human),[gene],"[b64807, SO_0001217]"
26366,A protein coding gene PAH in human.,HGNC_8582,PAH (human),[gene],"[b67084, SO_0001217]"
26367,A protein coding gene PRL in human.,HGNC_9445,PRL (human),[gene],"[b133804, SO_0001217]"
26368,A protein coding gene PTH in human.,HGNC_9606,PTH (human),[gene],"[b141039, SO_0001217]"


26370
<class 'pandas.core.frame.DataFrame'>
Int64Index: 26370 entries, 0 to 26369
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   description  22753 non-null  object  
 1   id           26370 non-null  object  
 2   name         26370 non-null  object  
 3   namespace    26370 non-null  category
 4   parents      26363 non-null  object  
dtypes: category(1), object(4)
memory usage: 1.0+ MB


None

CPU times: user 47.9 s, sys: 1.8 s, total: 49.7 s
Wall time: 48 s


In [12]:
# df.to_parquet("./proc/"+i.strip("./").replace(r"/","_")+".parquet",index=False)

In [13]:
df

,sourceDatabase,targetA,intA,intABiologicalRole,targetB,intB,intBBiologicalRole,speciesA,speciesB,count,scoring
0,intact,ENSG00000182667,Q9P121-3,unspecified role,ENSG00000106331,Q3KNR5,unspecified role,"{'mnemonic': 'human', 'scientific_name': 'Homo...","{'mnemonic': 'human', 'scientific_name': 'Homo...",3,0.560
1,intact,ENSG00000116171,P22307-2,unspecified role,ENSG00000105176,O94763-2,unspecified role,"{'mnemonic': 'human', 'scientific_name': 'Homo...","{'mnemonic': 'human', 'scientific_name': 'Homo...",1,0.350
2,intact,ENSG00000115138,P01189,unspecified role,ENSG00000150636,Q68D86,unspecified role,"{'mnemonic': 'human', 'scientific_name': 'Homo...","{'mnemonic': 'human', 'scientific_name': 'Homo...",3,0.560
3,intact,ENSG00000146530,Q8N2E2,unspecified role,ENSG00000204548,Q5J5C9,unspecified role,"{'mnemonic': 'human', 'scientific_name': 'Homo...","{'mnemonic': 'human', 'scientific_name': 'Homo...",1,0.350
4,intact,ENSG00000112640,Q14738-2,unspecified role,NaN,Q8N5D4,unspecified role,"{'mnemonic': 'human', 'scientific_name': 'Homo...","{'mnemonic': 'human', 'scientific_name': 'Homo...",1,0.350
...,...,...,...,...,...,...,...,...,...,...,...
12189708,string,ENSG00000215182,ENSP00000485659,unspecified role,ENSG00000173757,ENSP00000293328,unspecified role,"{'mnemonic': 'human', 'scientific_name': 'Homo...","{'mnemonic': 'human', 'scientific_name': 'Homo...",1,0.177
12189709,string,ENSG00000215182,ENSP00000485659,unspecified role,ENSG00000135838,ENSP00000258317,unspecified role,"{'mnemonic': 'human', 'scientific_name': 'Homo...","{'mnemonic': 'human', 'scientific_name': 'Homo...",2,0.206
12189710,string,ENSG00000215182,ENSP00000485659,unspecified role,ENSG00000205426,ENSP00000369349,unspecified role,"{'mnemonic': 'human', 'scientific_name': 'Homo...","{'mnemonic': 'human', 'scientific_name': 'Homo...",2,0.212
12189711,string,ENSG00000215182,ENSP00000485659,unspecified role,ENSG00000196611,ENSP00000322788,unspecified role,"{'mnemonic': 'human', 'scientific_name': 'Homo...","{'mnemonic': 'human', 'scientific_name': 'Homo...",2,0.229


In [14]:
# df.to_parquet("./proc/"+i.strip("./").replace(r"/","_")+".parquet",index=False)

Some dfs that fail:
* evidence
    * fastparquet OverflowError: value too large to convert to int
    * try :` engine='pyarrow'`
* drugWarnings

In [15]:
# df_mol = pd.read_parquet("./proc/molecule.parquet")
# display(df_mol.info(memory_usage=True))
# display(df_mol)

In [16]:
# ### SB can't handle list types - convert to categorical or string.. 
# for c in df_mol.select_dtypes("O").columns:
#     df_mol[c] = df_mol[c].astype(str).astype("category")
# df_mol.to_parquet("./proc/molecule_cat.parquet",index=False)

In [17]:
# df_hpo = pd.read_parquet("./proc/hpo.parquet")
# display(df_hpo.info(memory_usage=True))
# display(df_hpo)